In [1]:
import pandas as pd
# let's merge all the collected datasets
dataset_matches = pd.read_csv('../dataset/premier/lightgbm/lightgbm_base_odds_premier.csv')
dataset_matches = dataset_matches.sort_values(by=['match_year','match_month','match_day',
#                                                   'match_hour','match_round',
                                                  'home_team','away_team']).reset_index(drop=True)
dataset_matches = dataset_matches[['match_year','match_month','match_day','home_team','away_team','match_round','league_year','label']]
dataset_matches.head()

,match_year,match_month,match_day,home_team,away_team,match_round,league_year,label
0,2009,8,15,Aston Villa,Wigan,1,2009,2
1,2009,8,15,Blackburn,Manchester City,1,2009,2
2,2009,8,15,Bolton,Sunderland,1,2009,2
3,2009,8,15,Chelsea,Hull,1,2009,1
4,2009,8,15,Everton,Arsenal,1,2009,2


In [2]:
train = dataset_matches[:-100] 
validation = dataset_matches[-100:-50] 
test = dataset_matches[-50:] 
test.head()

,match_year,match_month,match_day,home_team,away_team,match_round,league_year,label
3750,2019,4,13,Southampton,Wolves,34,2018,1
3751,2019,4,13,Tottenham,Huddersfield,34,2018,1
3752,2019,4,14,Crystal Palace,Manchester City,34,2018,2
3753,2019,4,14,Liverpool,Chelsea,34,2018,1
3754,2019,4,15,Watford,Arsenal,34,2018,2


In [3]:
# predictions of the best model in STACKING.ipynb (60% accuracy)
predictions = [1,1,2,1,1,2,2,2,1,1,1,1,1,2,2,1,1,1,2,1,1,2,0,1,1,1,1,2,1,1,1,2,1,2,1,1,1,1,0,1,2,2,1,1,1,1,1,1,1,2]

In [6]:
# check accuracy
correct = 0
for ind,true,predicted in zip(test.index, test.label, predictions):
    if true == predicted:
        correct+=1
correct*100/test.shape[0]

60.0

In [10]:
#retrieve odds
odds_df = pd.read_csv('../dataset/premier/lightgbm/lightgbm_base_odds_premier.csv')
test_odds = pd.merge(test, odds_df, how='left')
test_odds = test_odds[['bet365_1','bet365_X','bet365_2']]
test_odds.head()

,bet365_1,bet365_X,bet365_2
0,2.70,3.20,2.87
1,1.20,7.50,17.00
2,13.00,6.75,1.25
3,1.72,3.90,5.25
4,3.70,3.70,2.05


In [30]:
result=[]
for p1,pX,p2 in zip(test_odds['bet365_1'], test_odds['bet365_X'], test_odds['bet365_2']):
    if min([p1,pX,p2]) == p1: result.append(1)
    elif min([p1,pX,p2]) == pX: result.append(0)
    else: result.append(2)
test_odds['preds'] = result

In [20]:
# bet 100$ on each match, using our best predictions
investement = 100
betting_return = 0
total_investement = 0
for pred, odd_1, odd_X, odd_2, label in zip(predictions, test_odds.bet365_1, 
                                            test_odds.bet365_X, test_odds.bet365_2, test.label):
    if pred == label:
        if pred == 1:
            betting_return += investement*odd_1
        elif pred == 2:
            betting_return += investement*odd_2
        else:
            betting_return += investement*odd_X
    total_investement += investement
print(f"Won {betting_return}$ with an investement of {total_investement}$. Overall return {betting_return-total_investement}$")

Won 6498.0$ with an investement of 5000$. Overall return 1498.0$


In [31]:
# bet 100$ on each match, using our best predictions
investement = 100
betting_return = 0
total_investement = 0
for pred, odd_1, odd_X, odd_2, label in zip(test_odds['preds'], test_odds.bet365_1, 
                                            test_odds.bet365_X, test_odds.bet365_2, test.label):
    if pred == label:
        if pred == 1:
            betting_return += investement*odd_1
        elif pred == 2:
            betting_return += investement*odd_2
        else:
            betting_return += investement*odd_X
    total_investement += investement
print(f"Won {betting_return}$ with an investement of {total_investement}$. Overall return {betting_return-total_investement}$")

Won 3923.0$ with an investement of 5000$. Overall return -1077.0$
